In [ ]:
! pip install python-mnist

In [ ]:
! pip install scikit-image

In [ ]:
! pip install imgaug

## Functions

In [ ]:
from torchvision.models.resnet import ResNet, BasicBlock
from torchvision.datasets import MNIST
from tqdm.autonotebook import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import inspect
import time
import torch
from torch import nn, optim
torch.manual_seed(1)
from torchvision.transforms import Compose, ToTensor, Normalize, Resize
from torch.utils.data import DataLoader 
import os
import torchvision
import torchvision.transforms as transforms
from collections import OrderedDict
from torch.optim import Adam, SGD
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
from skimage.metrics import structural_similarity as ssim
from PIL import Image
from imgaug import augmenters as iaa
import imgaug as ia
import matplotlib.pyplot as plt
import csv
import numpy as np
from torch_SinGAN import *

torch.cuda.set_device(0)

class TestResNet(ResNet):
    def __init__(self, channels=3, num_classes=10):
        super(TestResNet, self).__init__(BasicBlock, [2, 2, 2, 2], num_classes=num_classes)
        self.conv1 = torch.nn.Conv2d(channels, 64, 
            kernel_size=(7, 7), 
            stride=(2, 2), 
            padding=(3, 3), bias=False)

def calculate_metric(metric_fn, true_y, pred_y):
    # multi class problems need to have averaging method
    if "average" in inspect.getfullargspec(metric_fn).args:
        return metric_fn(true_y, pred_y, average="macro")
    else:
        return metric_fn(true_y, pred_y)

def make_data_loaders(X_train, y_train,batch_size=100):
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.25, random_state=42)
    data_transform = Compose([Resize((224, 224))])
    X_train = torch.from_numpy(X_train).type(torch.float)
    y_train = torch.from_numpy(y_train).type(torch.LongTensor)

    X_val = torch.from_numpy(X_val).type(torch.float)
    y_val = torch.from_numpy(y_val).type(torch.LongTensor)

    train_d = torch.utils.data.TensorDataset(X_train,y_train)
    train_loader = torch.utils.data.DataLoader(train_d, batch_size = batch_size, shuffle = False)

    val_d = torch.utils.data.TensorDataset(X_val,y_val)
    val_loader = torch.utils.data.DataLoader(val_d,batch_size = batch_size, shuffle = False)
    return train_loader, val_loader

def make_test_loarder(X_test, y_test, batch_size=100):
    X_test = torch.from_numpy(X_test).type(torch.float)
    y_test = torch.from_numpy(y_test).type(torch.LongTensor)
    test_d = torch.utils.data.TensorDataset(X_test,y_test)
    test_loader = torch.utils.data.DataLoader(test_d, batch_size = batch_size, shuffle = False)
    return test_loader

def print_scores(p, r, f1, a, batch_size):
    # just an utility printing function
    for name, scores in zip(("precision", "recall", "F1", "accuracy"), (p, r, f1, a)):
        print(f"\t{name.rjust(14, ' ')}: {sum(scores)/batch_size:.4f}")

def compute_test(model, X_test, y_test, c, batch_size=100):
    X_test = X_test.transpose(0,3,1,2)
    test_loader = make_test_loarder(X_test, y_test, batch_size=batch_size)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    criterion = torch.nn.CrossEntropyLoss()
    if torch.cuda.is_available():
        criterion = criterion.cuda()
    class_correct = 0
    class_total = 0
    model.eval()
    test_losses = 0
    precision, recall, f1, accuracy = [], [], [], []
    with torch.no_grad():
        for i, data in enumerate(test_loader):
            X, y = data[0].to(device), data[1].to(device)

            outputs = model(X) # this get's the prediction from the network

            test_losses += criterion(outputs, y)

            predicted_classes = torch.max(outputs, 1)[1] # get class from network's prediction
            
            # calculate P/R/F1/A metrics for batch
            for acc, metric in zip((precision, recall, f1, accuracy), 
                                    (precision_score, recall_score, f1_score, accuracy_score)):
                acc.append(
                    calculate_metric(metric, y.cpu(), predicted_classes.cpu())
                )
            class_correct += np.sum((predicted_classes.cpu().data.numpy().astype(int) == c) & (y.cpu().data.numpy().astype(int) == c))
            class_total += np.sum(y.cpu().data.numpy().astype(int) == c)
            
            
    print(f"test loss: {test_losses/len(test_loader)}")
    print_scores(precision, recall, f1, accuracy, batch_size)
    print(f"accuracy for class {c}: {class_correct/class_total}")
    print(class_correct, class_total)
    
    
def av_SSIM(images, other=None, pairs=1000):
    l = np.zeros(pairs)
    if other:
        ind_a = np.random.choice(list(range(images.shape[0])), size = pairs)
        ind_b = np.random.choice(list(range(other.shape[0])), size = pairs)
    else:
        ind_a = np.random.choice(list(range(images.shape[0])), size = pairs)
        ind_b = np.zeros(pairs, dtype=int)
        count = 0
        while count < pairs:
            ind_b[count] = np.random.choice(list(range(images.shape[0])), size = 1)[0]
            if ind_a[count] != ind_b[count]:
                count += 1
    
    for i in range(pairs):
        if other:
            l[i] = ssim(images[ind_a[i]], other[ind_b[i]], data_range=1, multichannel=True)
        else:
            l[i] = ssim(images[ind_a[i]], images[ind_b[i]], data_range=1, multichannel=True)
    
    return l.mean()
 
    
def get_classic_aug():
    ia.seed(42)
    seq = iaa.Sequential([
        iaa.Crop(percent=(0, 0.1)),
        iaa.Affine(
            scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
            translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
            rotate=(-25, 25),
            shear=(-8, 8)
        )
    ], random_order=True) 
    return seq
    
def train(model, train_loader, val_loader, epochs=5, learning_rate=0.01):
    def train_res(epoch):

        total_loss = 0

        # progress bar (works in Jupyter notebook too!)
        #progress = tqdm(enumerate(train_loader), desc="Loss: ", total=batches)
        
        progress = enumerate(train_loader)


        # ----------------- TRAINING  -------------------- 
        # set model to training
        model.train()

        for i, data in progress:
            X, y = data[0].to(device), data[1].to(device)

            # training step for single batch
            model.zero_grad()
            outputs = model(X)
            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()

            # getting training quality data
            current_loss = loss.item()
            total_loss += current_loss

            # updating progress bar
            #progress.set_description("Loss: {:.4f}".format(total_loss/(i+1)))

        # releasing unceseccary memory in GPU
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

        # ----------------- VALIDATION  ----------------- 
        val_losses = 0
        precision, recall, f1, accuracy = [], [], [], []

        # set model to evaluating (testing)
        model.eval()
        with torch.no_grad():
            for i, data in enumerate(val_loader):
                X, y = data[0].to(device), data[1].to(device)

                outputs = model(X) # this get's the prediction from the network

                val_losses += criterion(outputs, y)

                predicted_classes = torch.max(outputs, 1)[1] # get class from network's prediction

                # calculate P/R/F1/A metrics for batch
                for acc, metric in zip((precision, recall, f1, accuracy), 
                                       (precision_score, recall_score, f1_score, accuracy_score)):
                    acc.append(
                        calculate_metric(metric, y.cpu(), predicted_classes.cpu())
                    )
        
        if (epoch+1) % 5 == 0:
            print(f"Epoch {epoch+1}/{epochs}, training loss: {total_loss/batches}, validation loss: {val_losses/val_batches}")
            print_scores(precision, recall, f1, accuracy, val_batches)
        losses.append(total_loss/batches) # for plotting learning curve
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    start_ts = time.time()
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    losses = []
    batches = len(train_loader)
    val_batches = len(val_loader)

    criterion = torch.nn.CrossEntropyLoss()
    if torch.cuda.is_available():
        model = model.cuda()
        criterion = criterion.cuda()

    # loop for every epoch (training + evaluation)
    for epoch in range(epochs):
        train_res(epoch)

    print(f"Training time: {time.time()-start_ts}s")
    return model

def evaluate_ResNet(X_train, y_train,num_classes, epochs=5):
    X_train = X_train.transpose(0,3,1,2)
    train_loader, val_loader = make_data_loaders(X_train, y_train, batch_size=100)
    model = TestResNet(num_classes=num_classes)
    model = train(model, train_loader, val_loader, epochs=epochs)
    return model

def read_traffic_signs(rootpath, output_path):
    
    images = []
    labels = []
    # loop over all 42 classes
    for c in range(0,43):
        prefix = rootpath + '/' + format(c, '05d') + '/' # subdirectory for class
        gtFile = open(prefix + 'GT-'+ format(c, '05d') + '.csv') # annotations file
        gtReader = csv.reader(gtFile, delimiter=';') # csv parser for annotations file
        next(gtReader)
        # loop over all images in current annotations file
        for row in gtReader:
            im = Image.open(prefix + row[0])
            im = im.resize((56, 56))
            images.append(np.asarray(im))
            #im.save(os.path.join(output_path, "{}_{}.png".format(c, row[0][:row[0].index('.')])), "PNG")
            labels.append(row[7]) # the 8th column is the label
        gtFile.close()
        
    return np.asarray(images), np.asarray(labels)

def read_traffic_signs_test(rootpath, output_path):
    
    images = []
    labels = []

    prefix = rootpath + '/'
    gtFile = open('GT-final_test.csv') # annotations file
    gtReader = csv.reader(gtFile, delimiter=';') # csv parser for annotations file
    next(gtReader)
    # loop over all images in current annotations file
    for row in gtReader:
        im = Image.open(prefix + row[0])
        im = im.resize((56, 56))
        #im.save(os.path.join(output_path, "{}.png".format(row[0][:row[0].index('.')])), "PNG")
        images.append(np.asarray(im))
        labels.append(int(row[7])) # the 8th column is the label
    gtFile.close()
        
    return np.asarray(images), np.asarray(labels)

def eval_data_augmentation(dataset, c, ratio, X_train, y_train, X_test, y_test, num_classes=10, input_dir='input/', evaluation_method=['imbalanced','classic','BAGAN', 'SinGAN'],
                          epochs=5):
    class_indices = np.where(y_train == c)[0]
    if dataset == 'GTSRB':
        to_replace = np.random.choice(class_indices, len(class_indices)-ratio, replace=False)
    else:
        to_replace = np.random.choice(class_indices, int(ratio*class_indices.size), replace=False)
    print(len(to_replace))
    left = np.setdiff1d(class_indices, to_replace)
    shape = X_train.shape
    # load real training samples for minor class
    input_paths = list(filter(lambda x : x.startswith(f'{dataset}_{c}'), os.listdir(input_dir)))
    train_left = np.zeros(shape=(len(left), shape[1], shape[2], shape[3]))
    count = 0
    for path in input_paths:
        im = Image.open(input_dir+path)
        im = np.array(im)
        if im.shape != (shape[1],shape[2],shape[3]):
            im = im[:,:,:-1]
        train_left[count] = (im/255)
        count += 1
        if count == len(left): break
            
    for method in evaluation_method:
        if method == 'imbalanced':
            print(f'Evaluating on imbalanced data set on {dataset} class {c} ratio {ratio}')
            X_aug = X_train.copy()
            y_aug = y_train.copy()
            X_aug[left] = train_left
            X_aug = np.delete(X_aug, to_replace, axis=0)
            y_aug = np.delete(y_aug,to_replace, axis=0)
            model = evaluate_ResNet(X_aug, y_aug, num_classes, epochs=epochs)
            compute_test(model, X_test, y_test, c)
            del X_aug
            del y_aug
            
        if method == 'classic':
            print(f'Evaluating on classic augmentation on {dataset} class {c} ratio {ratio}')
            X_aug = X_train.copy()
            y_aug = y_train.copy()
            X_aug[left] = train_left
            seq = get_classic_aug()
            for i in range(to_replace.size):
                X_aug[to_replace[i]] = seq(image = X_aug[left][i % left.size])
            print(f'SSIM on {method} augmentation: {av_SSIM(X_aug[to_replace])}')
            model = evaluate_ResNet(X_aug, y_aug,num_classes, epochs=epochs)
            compute_test(model, X_test, y_test, c)
            del X_aug
            del y_aug
        
        if method == 'SinGAN':
            print(f'Evaluating on SinGAN augmentation on {dataset} class {c} ratio {ratio}')
            if dataset=='MNIST':
                simulated_imgs = simulated_imgs = generate_data(dataset,
                                                class_label = c, 
                                                layer_number = 6,
                                                additional_scale = 0,
                                                generate_size = to_replace.size, 
                                                model_size=len(left),
                                                generate_start_scale=0)
            else:
                simulated_imgs = simulated_imgs = generate_data(dataset,
                                                class_label = c, 
                                                layer_number = 5,
                                                additional_scale = 0,
                                                generate_size = to_replace.size, 
                                                model_size=len(left),
                                                generate_start_scale=0)

            print(f'SSIM on {method} augmentation: {av_SSIM(simulated_imgs)}')
            X_aug = X_train.copy()
            y_aug = y_train.copy()
            X_aug[left] = train_left
            X_aug[to_replace] = simulated_imgs
            del simulated_imgs
            model = evaluate_ResNet(X_aug, y_aug, num_classes, epochs=epochs)
            compute_test(model, X_test, y_test, c)
            del X_aug
            del y_aug
        
        if method == 'BAGAN':
            print(f'Evaluating on BAGAN augmentation on {dataset} class {c} ratio {ratio}')
            bagan_samples = np.zeros(shape=(len(to_replace), shape[1],shape[2],shape[3]))
            count = 0
            input_dir = f'./BAGANData/{dataset}/{ratio}/'
            for path in list(filter(lambda x: x.startswith('simulated'),os.listdir(input_dir))):
                im = Image.open(input_dir+path)
                im = np.array(im)
                bagan_samples[count] = im/255
                count += 1
                if count  == len(to_replace) : break
            print(f'SSIM on {method} augmentation: {av_SSIM(bagan_samples)}')
            X_aug = X_train.copy()
            y_aug = y_train.copy()
            X_aug[left] = train_left
            X_aug[to_replace] = bagan_samples
            del bagan_samples
            model = evaluate_ResNet(X_aug, y_aug, num_classes, epochs=epochs)
            compute_test(model, X_test, y_test, c)
            del X_aug
            del y_aug
        
        del model

def eval_architecture(dataset, c, ratio, X_train, y_train, X_test, y_test, num_classes, 
                      layer_number, addtional_scale, input_dir='input/', epochs=30):
    class_indices = np.where(y_train == c)[0]
    if dataset == 'GTSRB':
        to_replace = np.random.choice(class_indices, len(class_indices)-ratio, replace=False)
    else:
        to_replace = np.random.choice(class_indices, int(ratio*class_indices.size), replace=False)
    print(len(to_replace))
    left = np.setdiff1d(class_indices, to_replace)
    shape = X_train.shape
    # load real training samples for minor class
    input_paths = list(filter(lambda x : x.startswith(f'{dataset}_{c}'), os.listdir(input_dir)))
    train_left = np.zeros(shape=(len(left), shape[1], shape[2], shape[3]))
    count = 0
    for path in input_paths:
        im = Image.open(input_dir+path)
        im = np.array(im)
        if im.shape != (shape[1],shape[2],shape[3]):
            im = im[:,:,:-1]
        train_left[count] = (im/255)
        count += 1
        if count == len(left): break


        
    print(f'Evaluating on SinGAN augmentation on {layer_number}, {additional_scale}')
    simulated_imgs = generate_data(dataset,
                                    class_label = c, 
                                    layer_number = layer_number,
                                    additional_scale = additional_scale,
                                    generate_size = to_replace.size, 
                                    model_size=len(left),
                                    generate_start_scale=0)

    print(f'SSIM on augmentation: {av_SSIM(simulated_imgs)}')
    X_aug = X_train.copy()
    y_aug = y_train.copy()
    X_aug[left] = train_left
    X_aug[to_replace] = simulated_imgs
    del simulated_imgs
    model = evaluate_ResNet(X_aug, y_aug, num_classes, epochs=epochs)
    compute_test(model, X_test, y_test, c)
    del X_aug
    del y_aug

# Tuning Architecture Hyperparameters on GTSRB

In [ ]:
input_path_train = 'GTSRB/Training'
output_path_train = 'GTSRB/Training_png'

if not os.path.isdir(output_path_train):
    os.mkdir(output_path_train, 0o666)
    
X_train_GTSRB, y_train_GTSRB = read_traffic_signs(input_path_train, output_path_train)
y_train_GTSRB = y_train_GTSRB.astype("int")
X_train_GTSRB = X_train_GTSRB/255
print('Training Sample loaded')
classindex_sample = np.load("GTSRB_index.npy").flatten()
X_train_GTSRB = X_train_GTSRB[classindex_sample]
y_train_GTSRB = y_train_GTSRB[classindex_sample]

input_path_test = 'GTSRB/Final_Test/Images'
output_path_test = 'GTSRB/Final_Test/converted_png'

if not os.path.isdir(output_path_test):
    os.mkdir(output_path_test, 0o666)

X_test_GTSRB, y_test_GTSRB = read_traffic_signs_test(input_path_test, output_path_test)
X_test_GTSRB = X_test_GTSRB/255
print('Testing Sample loaded')

In [ ]:
for c in [13, 15, 33]:
    for layer_number, additional_scale in [(6,-2),(5,-1),(5,0),(6,-1)]:
        eval_architecture('GTSRB', c, 5, X_train_GTSRB, y_train_GTSRB, X_test_GTSRB, y_test_GTSRB, num_classes=43, 
                          layer_number=layer_number, addtional_scale=additional_scale, input_dir='input/', epochs=30)

# Evaluating on Datasets

## MNIST

Evaluation On Different Augmentation Approaches

In [ ]:
from mnist import MNIST
import numpy as np
mndata = MNIST('Data/MNIST')
X_train, y_train = mndata.load_training()
X_train = np.array(X_train).reshape(60000,28,28)
y_train = np.array(y_train)
X_test, y_test = mndata.load_testing()
X_test = np.array(X_test).reshape(10000,28,28)
y_test = np.array(y_test)

pre_X = X_train
X_train = np.zeros(shape=(60000,28,28,3))
for i in range(X_train.shape[0]):
    img = np.stack((pre_X[i],)*3, axis=-1)
    X_train[i] = img/255
del pre_X
pre_X_test = X_test
X_test = np.zeros(shape=(10000,28,28,3))
for i in range(X_test.shape[0]):
    img = np.stack((pre_X_test[i],)*3, axis=-1)
    X_test[i] = img/255
del pre_X_test


In [ ]:
c=4 #[2,7,4]
for ratio in [0.975, 0.99, 0.995]:
    eval_data_augmentation('MNIST', c, ratio, X_train, y_train, X_test, y_test, num_classes=10,
                      epochs=10)

In [ ]:
av_SSIM(X_train[y_train==c])

Evaluation on Full Dataset

In [ ]:
model = evaluate_ResNet(X_train, y_train, num_classes = 10, epochs=5)

In [ ]:
compute_test(model, X_test, y_test, c, batch_size=100)

In [ ]:
del X_train, y_train, X_test, y_test

## CIFAR 10

In [ ]:
from cifar10_web import cifar10
import numpy as np
from skimage import io as img

X_train_cifar10, y_train_cifar10, X_test_cifar10, y_test_cifar10 = cifar10(path=None)
y_train_cifar10 = np.array([np.argmax(a, axis=0) for a in y_train_cifar10])
y_test_cifar10 = np.array([np.argmax(a, axis=0) for a in y_test_cifar10])
X_train_cifar10 = X_train_cifar10.reshape(50000, 3, 32, 32).transpose(0,2,3,1)
X_test_cifar10 = X_test_cifar10.reshape(10000,3,32,32).transpose(0,2,3,1)

In [ ]:
c=3 #[8,9,3]
av_SSIM(X_train_cifar10[y_train_cifar10==c])

In [ ]:
model = evaluate_ResNet(X_train_cifar10, y_train_cifar10, num_classes=10, epochs=30)

In [ ]:
compute_test(model, X_test_cifar10, y_test_cifar10, c, batch_size=100)

In [ ]:
from cifar10_web import cifar10
import numpy as np
from skimage import io as img

X_train_cifar10, y_train_cifar10, X_test_cifar10, y_test_cifar10 = cifar10(path=None)
y_train_cifar10 = np.array([np.argmax(a, axis=0) for a in y_train_cifar10])
y_test_cifar10 = np.array([np.argmax(a, axis=0) for a in y_test_cifar10])
X_train_cifar10 = X_train_cifar10.reshape(50000, 3, 32, 32).transpose(0,2,3,1)
X_test_cifar10 = X_test_cifar10.reshape(10000,3,32,32).transpose(0,2,3,1)

for ratio in [0.975, 0.99, 0.995]:
    eval_data_augmentation('CIFAR10', 3, ratio, X_train_cifar10, y_train_cifar10, X_test_cifar10, y_test_cifar10, num_classes=10,
                      epochs=20)

In [28]:
del X_train_cifar10, y_train_cifar10, X_test_cifar10, y_test_cifar10

## GTSRB

In [ ]:
input_path_train = 'GTSRB/Training'
output_path_train = 'GTSRB/Training_png'

if not os.path.isdir(output_path_train):
    os.mkdir(output_path, 0o666)
    
X_train_GTSRB, y_train_GTSRB = read_traffic_signs(input_path_train, output_path_train)
y_train_GTSRB = y_train_GTSRB.astype("int")
X_train_GTSRB = X_train_GTSRB/255
print('Training Sample loaded')
classindex_sample = np.load("GTSRB_index.npy").flatten()
X_train_GTSRB = X_train_GTSRB[classindex_sample]
y_train_GTSRB = y_train_GTSRB[classindex_sample]

input_path_test = 'GTSRB/Final_Test/Images'
output_path_test = 'GTSRB/Final_Test/converted_png'

if not os.path.isdir(output_path):
    os.mkdir(output_path, 0o666)

X_test_GTSRB, y_test_GTSRB = read_traffic_signs_test(input_path_test, output_path_test)
X_test_GTSRB = X_test_GTSRB/255
print('Testing Sample loaded')

In [ ]:
c = 33  #[13,15,33]

for ratio in [5, 10, 15]:
    eval_data_augmentation('GTSRB', c, ratio, X_train_GTSRB, y_train_GTSRB, X_test_GTSRB, y_test_GTSRB, num_classes=43,
                      epochs=30,evaluation_method=['imbalanced','classic','BAGAN','SinGAN'])

In [ ]:
model = evaluate_ResNet(X_train_GTSRB, y_train_GTSRB, 43, epochs=20)
compute_test(model, X_test_GTSRB, y_test_GTSRB, c)
#del X_train_GTSRB, y_train_GTSRB, X_test_GTSRB, y_test_GTSRB

In [ ]:
len(np.where(y_test_GTSRB==c)[0])

In [ ]:
av_SSIM(X_train_GTSRB[y_train_GTSRB==c])